# Hydrangea Image Classification
---

# 0. Introduction

Although hydrangeas have felt familiar to me since childhood, I recently noticed that there are so many different varieties. Fascinated by their beauty and playfulness, I've decided to make a classification model of hydrangea's images. 

This project uses a dataset collected by scraping Google Images. As a beginner in ML/DS, I hope to share parts of my trial-and-error process with this limited dataset to receive feedback on my workflow and to potentially benefit other learners.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, foldernames, filenames in os.walk('/kaggle/input'): #path to input data
    for foldername in foldernames:
        print(os.path.join(dirname, foldername))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow==2.16.1

In [ ]:
!pip show tensorflow

In [ ]:
!pip show keras

# 1. Data Preparation

In [ ]:
import tensorflow
import keras
import numpy as np
from matplotlib import pyplot as plt

## 1-1 Load Data

In [ ]:
data_dir = '/kaggle/input/hydrangea-dataset-1'
data = keras.utils.image_dataset_from_directory(data_dir, label_mode="categorical")
class_names = data.class_names

In [ ]:
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [ ]:
print(len(batch) ) #returns 2: image and labels
print(batch[0].shape) #Images represented as numbpy arrays

In [ ]:
batch[0].min()

In [ ]:
batch[0].max()

## 1-2 Scale Data

In [ ]:
data = data.map(lambda x, y: (x/255, y))
scale_iterator = data.as_numpy_iterator().next()

In [ ]:
scale_iterator[0].min()

In [ ]:
scale_iterator[0].max()

## 1-3 Split Data

In [ ]:
len(data)

In [ ]:
train_size = int(len(data)*.7)+1
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
train_size+val_size+test_size 
# make sure this matches the "len(data)" to make full use of your data

In [ ]:
train_data = data.take(train_size)
validation_data = data.skip(train_size).take(val_size) #skip already used data
test_data = data.skip(train_size+val_size).take(test_size) #skip already used data

# 2. Model Building

## 2-1. Convolutional Neural Network

Having learned about CNN on [Computer Vision](https://www.kaggle.com/learn/computer-vision), I first tried to apply CNN on my own. The following is the entire code.

In [ ]:
from keras import layers
model=keras.Sequential([
    # Base
    layers.Conv2D(filters=64, kernel_size=(5, 5), padding='same',
                 activation='relu', input_shape=[256,256,3]),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Conv2D(filters=64, kernel_size=(5, 5), padding='same',
                 activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Conv2D(filters=64, kernel_size=(5, 5), padding='same',
                 activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Conv2D(filters=64, kernel_size=(5, 5), padding='same',
                 activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),

    # Head
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(5, activation='softmax')

])
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

model.summary()

## This is the interim model version used during the development process.
'''
history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=30,
)

# Plot learning curves
import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

print("Minimum validation loss: {}".format(history_frame['val_loss'].min()))
print("Maximum accuracy: {}".format(history_frame['val_accuracy'].max()))
'''

While I increased the number of layers to make a complex model, it didn't perform well. Thinking this was due to the small size of the dataset, I decided to try different methods.

## 2-2. Transfer Learning 

Realizing that my dataset was too small to train a model from scratch, I decided to use a pretrained model (VGG16) and implement data augmentation.

### 2-2-1 Initial Model

In [ ]:
from keras.applications import VGG16


pretrained_base = VGG16(
    include_top=False,
    input_shape=(256, 256, 3),
    weights='imagenet',
    pooling='max',
    classifier_activation='softmax',
)
pretrained_base.trainable = False

model = keras.Sequential([
    # Preprocessing
    layers.RandomFlip('horizontal'), # flip left-to-right
    layers.RandomRotation(factor=0.20),
    
    # Base
    pretrained_base,
    
    # Head
    layers.Flatten(),
    layers.Dense(128, activation='relu'),  # Increased size
    layers.Dense(64, activation='relu'),  # Additional dense layer
    layers.Dense(5, activation='softmax')  # Output layer for 5 classes
])

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

model.summary()

In [ ]:
# This is the interim model version used during the development process.
'''
history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=50,
    callbacks=[early_stopping],
)
# Plot learning curves
import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

print("Minimum validation loss: {}".format(history_frame['val_loss'].min()))
print("Maximum accuracy: {}".format(history_frame['val_accuracy'].max()))

'''

While training reached around 85% of validation accuracy with these methods, it was still volatile. To stabilize the training and aim for higher accuracy, I did the following:

1. Tried different learning rates:

* With a too small learning rate, the training started too slowly.
* With a too large learning rate, the training became too volatile towards the end.
* To address this, I introduced a learning rate schedule.

2. Tried different batch sizes:

* With a too small batch size, the training became too volatile.
* With a too large batch size, overfitting occurred.

3. Tried different numbers of epochs.

* Added data augmentation layers.

Along the way, both overfitting and underfitting occurred, so I added dropout layers but eventually commented them out. If the validation accuracy is consistently higher than the training accuracy, it indicates underfitting. Since this was the case, I removed the dropout layers.

### 2-2-2 Final Model

In [ ]:
pretrained_base = VGG16(
    include_top=False,
    input_shape=(256, 256, 3),
    weights='imagenet',
    pooling='max',
    classifier_activation='softmax',
)
pretrained_base.trainable = False

model = keras.Sequential([
    # Preprocessing
    layers.RandomFlip('horizontal'), # flip left-to-right
    layers.RandomRotation(factor=0.20),
    layers.RandomZoom(0.20),
    
    # Base
    pretrained_base,
    
    # Head
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),  # Add batch normalization
    #layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),  # Increased size
    layers.BatchNormalization(),
    #layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),  # Additional dense layer
    layers.BatchNormalization(),
    #layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')  # Output layer for 5 classes
])

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)


reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.00001
)


history = model.fit(
    train_data,
    validation_data=validation_data,
    batch_size=128,
    epochs=200,
    callbacks=[early_stopping, reduce_lr],
)

# Plot learning curves
import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

In [ ]:
print("Minimum validation loss: {}".format(history_frame['val_loss'].min()))
print("Maximum accuracy: {}".format(history_frame['val_accuracy'].max()))

# 3. Evaluation

In [ ]:
from keras.metrics import Precision, Recall, BinaryAccuracy

pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test_data.as_numpy_iterator():
    X,y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

print(f'Precision{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

### *Experiment with Images

In [ ]:
import cv2
img = cv2.imread('/kaggle/input/hydrangea-experiment/12780.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize = tensorflow.image.resize(img, (256, 256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
np.expand_dims(resize,0)
yhat = model.predict(np.expand_dims(resize/255,0))

In [ ]:
# Convert probabilities to class name
predicted_class_index = np.argmax(yhat)  # Get the index of the class with the highest probability
predicted_class_name = class_names[predicted_class_index]  # Map index to class name

print(f"Predicted class: {predicted_class_name}")
print(f"Probabilities: {yhat}")

# 4. Save the Model

In [ ]:
model.save('hydrangea_model.keras')

In [ ]:
loaded_model = keras.saving.load_model('/kaggle/working/hydrangea_model.keras')

In [ ]:
for batch in test_data.as_numpy_iterator():
    X,y = batch
    yhat = loaded_model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

print(f'Precision{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')